In [ ]:
import geopandas  as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import libpysal
from tobler.util import h3fy
from h3 import h3
#import descartes


import os
from os.path import isfile, join
from pathlib import Path
from os import listdir
import os
os.getcwd()

path =  Path(os.getcwd())
root = path.parent.absolute()

h3_level = 9

In [ ]:
path

In [ ]:
#hwy = gpd.read_file('/Users/josephschmit/Project/Capstone/data/Master_Plan_of_Highways/Master_Plan_of_Highways.shp')
gdf = gpd.read_file(root / 'X.data' / 'raw_data' / 'la_county_website_data' / 'County_Boundaries' / 'County_Boundaries.shp')
#get just inland la county
gdf = gdf.loc[gdf['OBJECTID'].eq(22)]


In [ ]:
gdf_hex = h3fy(gdf, h3_level)

In [ ]:
print(gdf_hex.columns)
gdf_hex.reset_index(inplace = True)
gdf_hex.shape

In [ ]:
gdf_hex.sample(3)

In [ ]:
import contextily as ctx
ax = gdf_hex.plot(markersize=0.01, alpha = .3, edgecolors= "black", figsize=(20, 25))

ctx.add_basemap(ax)
plt.show()

In [ ]:
gdf_hex[['hex_id', 'geometry']].to_file((root / 'X.data' / 'processed_data' / 'base_map_hex_all' / 'base_map_hex_all.shp'))